In [0]:
%tensorflow_version 2.x
import numpy as np
import pandas as pd
import string
from google.colab import drive
import scipy.sparse as sp
from scipy import stats
!pip install lightfm
from lightfm import LightFM
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
from tqdm.auto import tqdm
from sklearn.pipeline import Pipeline

In [0]:
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
ratings = pd.read_csv('/content/gdrive/My Drive/ratings.csv')
new_ratings = pd.read_csv('/content/gdrive/My Drive/new_ratings.csv')
ratings = pd.concat([ratings, new_ratings])

In [0]:
#A = new_ratings.values
#coder = {}
#k = 53426
#for i, line in enumerate(A):
#  if line[0] in coder:
#    A[i][0] = coder[line[0]]
#  else:
#    coder[line[0]] = k
#    A[i][0] = k
#    k += 1

In [0]:
#new_ratings = pd.DataFrame(A, columns=['user_id', 'book_id', 'rating'])

In [0]:
#new_ratings.to_csv('new_ratings.csv', index=False)

In [0]:
ratings['book_id'] -= 1
ratings['user_id'] -= 1
new_ratings['book_id'] -= 1
new_ratings['user_id'] -= 1

In [0]:
train_ratings, test_ratings = train_test_split(ratings, test_size=.3, random_state=42, shuffle=True)

In [0]:
#train_ratings['pos_ex'] = (train_ratings.rating >= 4).astype(float)
#train_ratings['neg_ex'] = -1 * (train_ratings.rating <= 3).astype(float)
#train_ratings['ex'] = train_ratings['pos_ex'] + train_ratings['neg_ex']

In [0]:
train_ratings.sort_values(by=['user_id', 'rating'], ascending=[True, False], inplace=True)
test_ratings.sort_values(by=['user_id', 'rating'], ascending=[True, False], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
train_ratings = sp.coo_matrix((train_ratings.rating.values, 
                               (train_ratings.user_id.values, train_ratings.book_id.values)))

In [0]:
books = pd.read_csv('/content/gdrive/My Drive/translated_books.csv')
titles = books['original_title']
gid = books['goodreads_book_id']

In [0]:
#books['std'] = np.sqrt((books.ratings_1 * (1 - books.average_rating) ** 2 + 
#                        books.ratings_2 * (2 - books.average_rating) ** 2 + 
#                        books.ratings_3 * (3 - books.average_rating) ** 2 + 
#                        books.ratings_4 * (4 - books.average_rating) ** 2 + 
#                        books.ratings_5 * (5 - books.average_rating) ** 2) / books.work_ratings_count)
#books['h'] = books['std'] * stats.t.ppf(1.95 / 2., books.work_ratings_count - 1)
#books['min_rating'] = books['average_rating'] - books['h']

In [0]:
#books['flag'] = pd.Series(books.ratings_count > 500000, dtype=np.int32, copy=True)
#books.sort_values(by=['flag', 'min_rating'], ascending=[False, False], inplace=True)
#books.drop(labels=['std', 'h'], axis=1, inplace=True)

In [0]:
#books.to_csv('sorted_trans_books.csv', index=False)

In [0]:
books.drop(labels=['ratings_count', 'work_ratings_count', 'work_text_reviews_count', 
                   'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 
                   'average_rating', 'original_title', 'автор', 'prep', 'sum', 'inv_sum',
                   'work_id', 'goodreads_book_id', 'book_id', 'isbn', 'isbn13', 'title', 
                   'image_url', 'small_image_url', 'best_book_id', 
                   'books_count', 'language_code'], axis=1, inplace=True)

In [0]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   authors                    10000 non-null  object 
 1   original_publication_year  9979 non-null   float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


In [0]:
books.fillna(books.original_publication_year.mean(), inplace=True)

In [0]:
authors = {}
for i, line in enumerate(books.authors.unique()):
  temp = list(map(lambda x: x.strip(), line.split(',')))
  for author in temp:
    if not authors.get(author):
      authors[author] = [0] * len(books)
    authors[author][i] = 1

In [0]:
pref = set()
for key in authors.keys():
  s = sum(authors[key])
  if s > 1:
    pref.add(key)
pref.add('Others')

In [0]:
authors['Others'] = [0] * len(books)
d = authors.copy()
for author in d.keys():
  if author not in pref:
    for i in range(len(books)):
      if d[author][i]:
        authors['Others'] = 1
    del authors[author]

In [0]:
authors = pd.DataFrame.from_dict(authors)

In [0]:
books.drop(labels=['authors'], axis=1, inplace=True)

In [0]:
X = books.values
X = minmax_scale(X)

In [0]:
books = pd.DataFrame(X, columns=books.columns)

,original_publication_year
0,0.997611
1,0.994691
2,0.996814
3,0.984869
4,0.975577


In [0]:
books = pd.concat((books, authors), axis=1)

In [0]:
#books = pd.read_csv('/content/gdrive/My Drive/auth_books.csv')
book_features = sp.csr_matrix(books)

In [0]:
tags = pd.read_csv('/content/gdrive/My Drive/tags.csv')

In [0]:
def delete_punctuation(text):
  exclude = string.punctuation + "«»\n-"
  table = str.maketrans(exclude, " " * len(exclude))
  return text.translate(table).lower()

In [0]:
def format_str(text):
  words = [word for word in delete_punctuation(text).split()]
  return ' '.join(words)

In [0]:
t = {}
for i, line in enumerate(tqdm(tags.values)):
  if type(line[1]) == str:
    t[line[0]] = format_str(line[1])

In [0]:
book_tags = pd.read_csv('/content/gdrive/My Drive/book_tags.csv')

In [0]:
g = {}
book_tags.head(50)
for i, line in enumerate(book_tags.values):
  if line[0] in g:
    if line[1] in t:
      if line[2] >= 500:
        g[line[0]] += ' ' + t[line[1]]
      else:
        del t[line[1]]
  else:
    if line[1] in t:
      if line[2] >= 500:
        g[line[0]] = t[line[1]]

In [0]:
t = []
for i in gid:
  if i in g:
    t.append(g[i])
  else:
    t.append('')

In [0]:
vectorizer = CountVectorizer()
vectorized_tags = vectorizer.fit_transform(t)
transformer = TfidfTransformer()
transformed_tags = transformer.fit_transform(vectorized_tags)

In [0]:
transformed_tags

<10000x246 sparse matrix of type '<class 'numpy.float64'>'
	with 1683 stored elements in Compressed Sparse Row format>

In [0]:
book_features = sp.hstack((book_features, transformed_tags))

In [0]:
book_features

<10000x1089 sparse matrix of type '<class 'numpy.float64'>'
	with 24046 stored elements in COOrdinate format>

In [0]:
model = LightFM(no_components=50, loss='warp', random_state=42)
history = []

In [0]:
n = 150 #(150)
for i in range(n):
  model.fit_partial(train_ratings, item_features=book_features, num_threads=cpu_count(), epochs=1)
  pred_ratings = pd.DataFrame(test_ratings)
  pred_ratings['probs'] = model.predict(test_ratings.user_id.values, 
                                        test_ratings.book_id.values, 
                                        item_features=book_features, num_threads=cpu_count())
  pred_ratings.sort_values(by=['user_id', 'probs'], ascending=[True, False], inplace=True)
  history.append(np.sqrt(mean_squared_error(pred_ratings.rating.values, test_ratings.rating.values)))
  print(history[i])

1.2078613762593366
1.207671582383971
1.2070474997838834
1.2063694663777116
1.2060938919759328
1.20565450726013
1.2054953642471926
1.205468992591005
1.2052306962046226
1.2049798557134834
1.2045409906710942
1.2044053175426472
1.2040866802745618
1.2036396123238586
1.2034815931599592
1.2035163498878327
1.2031469503690078
1.2028140692238571
1.2024217261015646
1.2023335936103876
1.2021772585093153
1.201935525351789
1.2020904999936053
1.201814408503403
1.2015999878294736
1.2015340759517363
1.201381350734436
1.2010205821689355
1.2009314180251143
1.200838531668523
1.2007869766401182
1.2005863089686313
1.2005760888841333
1.2002104313487003
1.200070085916501
1.2001918434926286
1.2001239953748264
1.1999924706328537
1.1999246112417823
1.1997735405968273
1.1997688919676353
1.1998711576495387
1.1996745209007107
1.19939926829424
1.1993420710597373
1.1993527666849149
1.1994160084322527
1.1994936610168707
1.1995415520208352
1.1994583227394175
1.1994880813580666
1.1991509288044462
1.1993774127624128
1.19

In [0]:
u = {}
for line in new_ratings.values:
  if u.get(line[0]):
    u[line[0]].add(line[1])
  else:
    u[line[0]] = set()
    u[line[0]].add(line[1])

In [0]:
pred_u = []
pred_b = []
for key in u.keys():
  k = 0
  for i in range(10000):
    if k == 50:
      break
    if i not in u[key]:
      pred_u.append(int(key))
      pred_b.append(i)
      k += 1

In [0]:
preds = pd.DataFrame(data=zip(pred_u, pred_b), index=None, columns=['user_id', 'book_id'])
preds.head()

,user_id,book_id
0,53425,2
1,53425,3
2,53425,4
3,53425,5
4,53425,6


In [0]:
preds['score'] = model.predict(preds.user_id.values, preds.book_id.values, 
                      item_features=book_features)

In [0]:
preds.sort_values(by=['user_id', 'score'], ascending=[True, False], inplace=True)

In [0]:
preds

,user_id,book_id,score
9,53425,12,-3.769799
22,53425,26,-3.948902
17,53425,20,-3.965206
13,53425,16,-4.017804
0,53425,2,-4.237250
...,...,...,...
948,53443,50,-5.629990
900,53443,0,-5.630939
930,53443,30,-5.631886
911,53443,11,-5.633782


In [0]:
preds.drop(labels=['score'], axis=1, inplace=True)

In [0]:
preds.to_csv('predictions.csv', index=False)

In [0]:
print(u[53426])

{0, 1, 513, 7, 2186, 46, 591, 47, 497, 1040, 16, 80, 22, 54, 7067, 31}


In [0]:
sum(titles == "Harry Potter and the Philosopher's Stone")

1

In [0]:
for line in preds.values:
  if line[0] == 53426:
    print(titles[line[1]])

Nineteen Eighty-Four
The Kite Runner 
Harry Potter and the Order of the Phoenix
Harry Potter and the Half-Blood Prince
Twilight
The Great Gatsby
The Secret Life of Bees
Animal Farm: A Fairy Story
O Alquimista
Water for Elephants
The Hobbit or There and Back Again
Män som hatar kvinnor
Jane Eyre
The Fault in Our Stars
Mockingjay
The Da Vinci Code
Lord of the Flies 
Pride and Prejudice
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
A Game of Thrones
The Lightning Thief
Gone Girl
Eclipse
Breaking Dawn
Fifty Shades of Grey
Where the Sidewalk Ends: The Poems and Drawings of Shel Silverstein
The Notebook
The Adventures of Huckleberry Finn
An Excellent conceited Tragedie of Romeo and Juliet
Little Women
The Lion, the Witch and the Wardrobe
Charlotte's Web
 The Fellowship of the Ring
To Kill a Mockingbird
The Hitchhiker's Guide to the Galaxy
The Giver
Memoirs of a Geisha
Harry Potter and the Prisoner of Azkaban
Angels & Demons 
Harry Potter and the Goblet of Fire
Life of Pi
The 

In [0]:
for val in u[53426]:
  print(titles[val])

The Hunger Games
Harry Potter and the Philosopher's Stone
The Adventures of Sherlock Holmes
The Catcher in the Rye
Мы
The Book Thief
Мастер и Маргарита
Fahrenheit 451
Война и миръ
Идиот
Catching Fire
The Glass Castle
Harry Potter and the Chamber of Secrets
Brave New World
Martin Eden
Of Mice and Men 


In [0]:
R = pd.pivot_table(
    ratings[-1000000:],
    values='rating',
    index='user_id',
    columns='book_id').fillna(0)

In [0]:
from scipy.linalg import svd

In [0]:
U, sigma, V = svd(R)
print(ratings.shape, U.shape, sigma.shape, V.shape)

(5976616, 3) (30745, 30745) (9999,) (9999, 9999)


In [0]:
Sigma = np.zeros((30745, 9999))
Sigma[:9999, :9999] = np.diag(sigma)

new_ratings = U.dot(Sigma).dot(V)
print(sum(sum((new_ratings - R.values) ** 2)))

6.939214776909276e-22


In [0]:
K = 100
####### Здесь ваш код ##########
sigma[K:] = 0
Sigma = np.zeros((30745, 9999))
Sigma[:9999, :9999] = np.diag(sigma)

In [0]:
new_ratings = U.dot(Sigma).dot(V)
print(sum(sum((new_ratings - R.values) ** 2)))
print(sum(sum((R.values.mean() - R.values) ** 2)))

11892145.287572963
16275200.248298053


In [0]:
R.shape

(30745, 9999)

In [0]:
s = 0
k = 0
for i in range(R.shape[0]):
  for j in range(R.shape[1]):
    if R.values[i][j] != 0:
      k += 1
      s += (new_ratings[i][j] - R.values[i][j]) ** 2
print((s / k) ** .5)

3.2203191217624547


In [0]:
new_ratings = pd.DataFrame(new_ratings, index=R.index, columns=R.columns)

In [0]:
new_ratings.head()

book_id,2,3,4,5,7,8,9,10,11,13,14,15,18,19,21,22,23,24,25,26,27,28,29,32,33,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,9903,9905,9908,9910,9911,9919,9921,9922,9923,9924,9925,9927,9928,9934,9935,9936,9940,9946,9948,9960,9961,9962,9963,9966,9967,9972,9973,9974,9977,9978,9981,9984,9985,9988,9990,9991,9995,9997,9998,10000
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.118873,0.025950,0.043151,-0.010069,0.031581,-0.011228,-0.041136,-0.026962,0.044509,-0.005319,0.053111,0.080638,0.081706,-0.050763,0.028902,-0.060569,0.003920,0.003825,-0.053743,-0.020463,0.011935,-0.037425,-0.005986,0.002126,-0.010860,0.008972,0.041431,0.041986,0.004052,0.206344,-0.109591,0.081307,0.086869,-0.061023,0.065307,-0.017077,-0.080101,2.117745,-0.046517,-0.025589,...,0.004906,0.057404,-0.001746,-0.009045,0.000222,-0.041600,-0.022317,-0.015665,-0.009922,-0.011230,0.027193,-0.048825,0.015072,0.046339,-0.026612,0.056204,-0.016763,0.041929,0.050706,0.033693,-0.017055,0.050284,-0.010327,-0.178615,0.044974,-0.107510,-0.008402,-0.003888,0.008257,0.002403,0.031812,-0.039181,-0.017752,-0.009821,-0.021879,-0.139415,0.123521,-0.034000,-0.029261,0.024818
2,-0.175547,0.087012,0.029240,0.014883,0.066045,-0.005866,0.006834,-0.015018,-0.067761,0.017300,-0.054024,-0.067755,0.125785,0.063315,0.049511,-0.027222,0.032723,0.023453,-0.034734,4.018244,-0.065814,0.015061,0.068026,-0.106932,3.928893,-0.096206,-0.034047,0.004566,-0.001566,0.042330,0.150782,0.066057,-0.041849,0.008470,0.115561,-0.016542,0.087322,0.188391,0.080681,-0.086860,...,-0.003688,0.010270,-0.001627,-0.007368,-0.014420,0.018328,0.114133,-0.053209,-0.025296,0.052431,0.008503,-0.059896,0.006541,-0.041553,0.009742,0.027126,0.004777,-0.008061,0.012960,-0.011762,-0.108274,0.029513,0.116130,0.048297,0.005650,0.093807,-0.057495,-0.085689,0.085181,0.040174,0.024080,0.002140,0.018651,0.030585,0.017136,-0.013169,0.114477,-0.024318,-0.029602,0.039011
3,-0.076800,0.061750,2.906562,0.039505,0.045817,-0.056309,0.044602,-0.037060,0.025712,1.967249,0.040849,0.003103,0.007560,-0.061488,0.058079,-0.007734,0.059240,-0.013354,0.019889,0.001616,-0.056248,1.972587,-0.093446,0.059590,0.017130,0.033035,0.020413,0.111989,-0.095695,0.019467,-0.112582,-0.077757,-0.046618,0.037023,0.061838,-0.016999,0.096638,-0.010761,0.025964,-0.033686,...,0.004258,-0.045630,0.002772,0.053749,-0.014036,0.003877,0.058278,-0.024296,0.007487,-0.039226,-0.012258,0.151538,0.051731,0.023644,0.028916,-0.064602,-0.003579,0.214558,-0.005329,0.008117,0.055249,0.569331,-0.012862,0.667533,-0.046181,-0.090713,0.021581,-0.073170,0.117433,-0.019651,-0.143925,-0.019615,-0.011369,-0.032290,-0.057586,-0.100703,-0.013705,0.023373,-0.009029,0.071271
4,4.989835,-0.080632,3.957341,3.983841,-0.084418,4.008420,-0.070578,5.036412,3.998674,3.985042,0.043348,0.022940,4.951395,0.027393,5.056889,0.006804,5.039360,5.055656,-0.032870,3.003735,4.976314,3.020582,0.011472,4.012840,2.967699,4.992621,5.017365,-0.001561,-0.011050,-0.126270,-0.057724,0.071386,5.012523,0.030156,0.020588,4.009378,-0.055755,0.070100,4.022090,0.042386,...,-0.024348,-0.015035,-0.002892,0.012924,-0.046681,0.026094,-0.009179,-0.019047,-0.016411,0.007219,0.023828,0.026674,0.031789,0.005933,-0.006435,0.131692,0.009215,0.082259,-0.002659,-0.033524,-0.051693,-0.140710,0.018269,-0.071050,0.008105,0.061557,-0.064690,0.124756,-0.025569,-0.079722,0.037377,-0.020497,0.014343,-0.065983,0.013516,0.089898,0.016141,-0.015694,0.005196,0.045544
6,0.036115,0.001089,-0.020454,-0.000488,-0.020916,0.011715,0.009623,0.014306,0.000919,0.016817,0.009925,-0.015558,-0.007579,-0.006118,-0.007462,0.002051,-0.031581,0.002679,-0.010708,-0.011626,0.001460,-0.012542,-0.005462,0.014350,0.010758,0.023715,-0.000330,-0.008818,0.032917,0.009559,0.044787,0.003463,-0.000654,-0.020890,0.012820,-0.002698,-0.014307,0.021571,0.027381,0.007150,...,0.000566,0.006731,-0.000185,0.003986,-0.004597,0.009602,0.005820,-0.005316,-0.000943,-0.003389,0.005063,-0.018564,0.001125,-0.005891,-0.002671,-0.020682,0.0001